In [13]:
import joblib
import sys
from os.path import abspath
import numpy as np
from sklearn.preprocessing import StandardScaler
if abspath('../../utils/') not in sys.path:
    sys.path.append(abspath('../../utils/'))
from utils import GetExtent, ventana

features = ['dem', 'vegh', 'slope', 'aspect', 'curvature']

In [2]:
datos = joblib.load('../../processed/datos.joblib')

### Print the Shapes of the arrays found in the dictionary.

In [13]:
for llave, d in datos.items():
    print(llave, d.shape)

snowdepth (17002, 17569)
vegh (17002, 17569, 1)
dem (17002, 17569, 1)
slope (17002, 17569, 1)
aspect (17002, 17569, 1)
curvature (17002, 17569, 1)


In [14]:
for llave, d in datos.items():
    print(d.shape)

(17002, 17569)
(17002, 17569, 1)
(17002, 17569, 1)
(17002, 17569, 1)
(17002, 17569, 1)
(17002, 17569, 1)


## Generate the X and Y for Keras

In [16]:
X = np.stack([d for llave, d in datos.items() if llave != 'snowdepth'], axis=-1)
Y = datos['snowdepth']

This is kind of __hacky__ and needs to be reviewed

In [29]:
X = np.swapaxes(X, -2, -1)

X = X.squeeze(axis=-1)

Y = np.expand_dims(Y, axis=-1)

In [31]:
print(X.shape)
print(Y.shape)

(17002, 17569, 5)
(17002, 17569, 1)


In [32]:
Xlist = ventana(signal=X, kernel_size=(1000, 1000), stride=(1000, 1000))
Ylist = ventana(signal=Y, kernel_size=(1000,1000), stride=(1000,1000))

### Joblib the list of windows for Future use.

In [33]:
joblib.dump([Xlist, Ylist], '../../processed/XY.joblib')  

['../../processed/XY.joblib']

In [2]:
XX, *YY = joblib.load('../../processed/XY.joblib')

In [8]:
XX = np.stack(XX, axis=0)
YY = np.stack(YY, axis=0)

In [9]:
XX.shape

(289, 1000, 1000, 5)

In [ ]:
Xout = {}
Yout = {}
UTMEout = {}
UTMNout = {}
ixs = {}
train_per = .20
# Permuter ---------------------------
train_size = np.int(np.ceil(train_per * XX.shape[0]))
permutation = np.random.permutation(XX.shape[0])
ixs['train'] = permutation[:train_size]
ixs['test'] = permutation[train_size:]

# Trainset ------------------------
Xout['train'] = XX[ixs['train'], :, :, :]
Yout['train'] = YY[ixs['train'], :, :, :]
# UTMEout['train'] = UTME[ixs['train'], :, :, :]
# UTMNout['train'] = UTMN[ixs['train'], :, :, :]

# Testset -------------------------
Xout['test'] = XX[ixs['test'], :, :, :]
Yout['test'] = YY[ixs['test'], :, :, :]

# UTMEout['test'] = UTME[ixs['test'], :, :, :]
# UTMNout['test'] = UTMN[ixs['test'], :, :, :]

# Scale ------------------
scaler = StandardScaler()
Xtr_tmp = scaler.fit_transform(Xout['train'].reshape(-1, XX.shape[-1]))
Xout['train'] = Xtr_tmp.reshape(Xout['train'].shape)
Xte_tmp = scaler.transform(Xout['test'].reshape(-1, XX.shape[-1]))
Xout['test'] = Xte_tmp.reshape(Xout['test'].shape)